In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from functools import partial
import nltk
import pickle as pk
import torch
from context_enforcement.models.context_enforcer import compute_context_boundary
from context_enforcement.trainers.train_t5 import model_init
from context_enforcement.data.common import create_text_tokenizer, SmartCollator
from context_enforcement.trainers.common import get_dataset_specified_tasks
from pytorch_lightning import seed_everything

import sys

seed_everything(1376)

/home/nlplab/anaconda3/envs/development/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 1376
Global seed set to 1376
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 1376


1376

In [7]:
configs = {}
configs["model_base"] = "t5-base"
configs["task_type"] = "xsum"
configs["context_max_len"] = 512
configs["task_type"] = "xsum"
from argparse import Namespace
configs = Namespace(**configs)
#configs= pk.load(open("trained_models/t5models/t5-base_model_ce/train_args.ap",'rb'))
context_max_len=configs.context_max_len
context_sampling_bounds=(0.1, 0.45)

TypeError: namedtuple() missing 1 required positional argument: 'field_names'

In [8]:
tokenizer = create_text_tokenizer(configs.model_base)

task_dataset_gen = get_dataset_specified_tasks(configs.task_type)

train_dataset = None
eval_dataset = None
test_dataset = None
if task_dataset_gen is not None:
    raw_dataset = task_dataset_gen(tokenizer=tokenizer, )
    train_dataset = raw_dataset['train']
    eval_dataset = raw_dataset['validation']
    test_dataset = raw_dataset['test']

model_builder = model_init(
        vocab_size=len(train_dataset.tokenizer),
        model_base=configs.model_base,
        is_baseline=False,
        context_max_len = context_max_len,
        context_sampling_bounds = context_sampling_bounds,
    )

/home/nlplab/anaconda3/envs/development/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Found cached dataset xsum (/home/nlplab/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)
100%|██████████| 3/3 [00:00<00:00, 455.57it/s]


In [9]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")#

In [10]:
device

device(type='cuda')

In [11]:
generator = model_builder()
train_model_path = "trained_models/t5models/t5-base_model_ce2/checkpoint-17004/pytorch_model.bin"
state_dict = torch.load(train_model_path)
generator.load_state_dict(state_dict)

Some weights of T5ContextEnforcementSeqModel were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.block.1.context_enforcer_layer_norm.bias', 'encoder.block.1.context_enforcer._left_mha.out_proj.weight', 'encoder.block.0.context_enforcer._right_mha.in_proj_weight', 'encoder.block.8.context_enforcer._left_mha.in_proj_weight', 'encoder.block.8.context_enforcer._right_mha.out_proj.bias', 'encoder.block.9.context_enforcer_layer_norm.weight', 'encoder.block.1.context_enforcer._right_mha.out_proj.weight', 'encoder.block.4.context_enforcer._wlc.bias', 'encoder.block.10.context_enforcer._right_mha.out_proj.bias', 'encoder.block.7.context_enforcer._right_mha.in_proj_weight', 'encoder.block.7.context_enforcer._left_mha.in_proj_weight', 'encoder.block.1.context_enforcer._wlc.bias', 'encoder.block.2.context_enforcer._left_mha.in_proj_weight', 'encoder.block.3.context_enforcer._wrc.bias', 'encoder.block.11.context_enforcer._right_mha.in_proj_bias', 'encoder.

<All keys matched successfully>

<All keys matched successfully>

In [12]:
te1= test_dataset[80]
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)
context_boundary = compute_context_boundary(512,context_max_len=200)
context_boundary

(132, 332)

In [7]:
seed_everything(1376)

Global seed set to 1376


1376

In [20]:
input_ids=torch.randint(1,300,(1,512),device=device)
attention_mask=torch.ones_like(torch.randint(1,300,(1,512)),device=device)

tensor([[226, 155, 192]])

In [15]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids,
                attention_mask=b_input_mask,
                #context_boundary =context_boundary,
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=150,
        early_stopping=True,
        use_cache=True,
        num_beams=10,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

['A man has been arrested on suspicion of murder after a man was stabbed to death in a street attack.']

In [13]:
test_dataset.tokenizer.decode(te1.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True)

'A 34-year-old woman who was injured in an attack by a knifeman on a Swiss train has died in hospital, police say.'

In [11]:
te1.input_ids.shape

torch.Size([1258])

In [58]:
print(test_dataset.tokenizer.decode(te1.input_ids,clean_up_tokenization_spaces=True,skip_special_tokens=True))

Media playback is not supported on this device
The Olympic silver medallist accused the organisation of "ageism" and having "zero regard" for her welfare.
She is the latest high-profile cyclist to come forward after Jess Varnish, Nicole Cooke and Emma Pooley criticised the World Class programme.
Houvenaghel told the BBC she felt "vindicated" by a leaked draft report detailing British Cycling's failures.
The report said British Cycling "sanitised" its own investigation into claims former technical director Shane Sutton used sexist language towards Varnish, who went public last April about her treatment.
British Cycling subsequently admitted it did not pay "sufficient care and attention" to the wellbeing of staff and athletes at the expense of winning medals, an approach Houvenaghel attested to in her BBC interview.
Both Sutton and predecessor Sir Dave Brailsford have now left British Cycling.
Houvenaghel, 42, spoke to BBC Sport during its State of Sport week, which on Thursday examines 

In [31]:
boundary_mask

(360, 610)

In [8]:
from torch.utils.data import DataLoader,SequentialSampler
import tqdm
test_data_loader = DataLoader(test_dataset,batch_size=12,
                              sampler= SequentialSampler(test_dataset),
                              collate_fn= SmartCollator(
            pad_token_id=train_dataset.tokenizer.pad_token_id, max_len=800,
            context_max_len=300,
            context_sampling_bounds=(0.1, 0.45)
        )
                              )

output_summaries2 =[]
for batch in tqdm.tqdm(test_data_loader):
    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    
    seq_len = b_input_ids.shape[1]
    
    boundary_mask =  batch.get("boundary",)
    bb=generator.generate(input_ids=b_input_ids,
            attention_mask=b_input_mask,
            num_beams=10,
            do_sample=False,
            num_return_sequences=1,
            max_length=140)
    sentences = test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)
    output_summaries2+=sentences

100%|██████████| 945/945 [35:53<00:00,  2.28s/it]


(360, 470)

In [9]:
#context_enforcement/data/common.py
from context_enforcement.data.common import write_to_file
write_to_file(output_summaries2[:len(test_dataset)], "outputs/attempt1/t5_baseline")

Done


In [10]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

In [11]:
targets[690],output_summaries2[690]

('Musician Koffi Olomide has been taken into custody in the Democratic Republic of Congo, days after he was deported from Kenya for allegedly kicking one of his dancers at an airport in Nairobi.',
 'Congolese singer Olomide has been detained by police in the Democratic Republic of Congo after he was filmed assaulting a dancer.')

In [12]:
import evaluate
metrics = evaluate.combine(['bleu','meteor',"rouge"])

[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
scores4 = metrics.compute(predictions=output_summaries2,references=targets)
scores4

{'bleu': 0.10229038581666045,
 'precisions': [0.3928173359103983,
  0.14167718301959548,
  0.07141177501233384,
  0.039391875456093406],
 'brevity_penalty': 0.9144670367817925,
 'length_ratio': 0.9179248030409859,
 'translation_length': 239549,
 'reference_length': 260968,
 'meteor': 0.32270879955148984,
 'rouge1': 0.3723759012640018,
 'rouge2': 0.15332988714608908,
 'rougeL': 0.29900944005710295,
 'rougeLsum': 0.2990488997322651}

In [13]:
scores = metrics.compute(predictions=output_summaries2,references=targets)

In [1]:
scores

NameError: name 'scores' is not defined

In [30]:
scores

{'bleu': 0.08190144303338862,
 'precisions': [0.39916558741021463,
  0.13484576712224944,
  0.06490478666102122,
  0.03461400195153127],
 'brevity_penalty': 0.7810193894484071,
 'length_ratio': 0.8018247587808374,
 'translation_length': 209249,
 'reference_length': 260966,
 'meteor': 0.28558238733460617,
 'rouge1': 0.34475899874216664,
 'rouge2': 0.13444853158452552,
 'rougeL': 0.279360220097055,
 'rougeLsum': 0.27932587220115546}

In [39]:
scores2

{'bleu': 0.08004077243766287,
 'precisions': [0.39668147524032016,
  0.1326978952590178,
  0.06327462141553002,
  0.03340461786535229],
 'brevity_penalty': 0.7793378458842136,
 'length_ratio': 0.8004414368155238,
 'translation_length': 208888,
 'reference_length': 260966,
 'meteor': 0.28278544337874784,
 'rouge1': 0.34230456443341767,
 'rouge2': 0.13258619227287083,
 'rougeL': 0.27739509890876,
 'rougeLsum': 0.2774115466752346}

In [28]:
scores3

{'bleu': 0.11312982003210258,
 'precisions': [0.43367935409457903,
  0.170965703452634,
  0.09041255378385218,
  0.051220088499377064],
 'brevity_penalty': 0.8310753377221631,
 'length_ratio': 0.8438570541756397,
 'translation_length': 220218,
 'reference_length': 260966,
 'meteor': 0.3379248069191188,
 'rouge1': 0.3902876664001791,
 'rouge2': 0.17375564911724187,
 'rougeL': 0.3204999917403623,
 'rougeLsum': 0.3205687202163906}

In [35]:
scores4

{'bleu': 0.11146082917260128,
 'precisions': [0.4317755562839142,
  0.1685194251648763,
  0.08904895332074475,
  0.050525125222168364],
 'brevity_penalty': 0.8286310630214466,
 'length_ratio': 0.8417648276020631,
 'translation_length': 219672,
 'reference_length': 260966,
 'meteor': 0.334512237524537,
 'rouge1': 0.3879467178046803,
 'rouge2': 0.17119907794549927,
 'rougeL': 0.31801217096792334,
 'rougeLsum': 0.31809606706132665}